# Chapter 6.2 - Spark SQL continued

For this section we will analyze a very large dataset from the Department of Transportation. I have already converted the data to the parquet format discussed in class, but if you don't believe me that it has benefits, let's check out some stats:

Here is the original data as I downloaded it without any modification other than I unzipped it.

In [1]:
!du -sh /disk/airline-data

44G	/disk/airline-data


Let's take a look at the data after I processed it.

In [2]:
!du -sh /disk/airline-data-processed

3.7G	/disk/airline-data-processed


Well I don't know about you, but that seems amazing :) Here is the original compressed file size. It is important to realize that while the .tar.gz file is "small" at 4.7 GB, we can't access it with Spark or any other program without uncompressing it. But we can do that with the parquet files!

In [3]:
!du -sh /disk/airline-data.2003-2018.tar.gz

4.7G	/disk/airline-data.2003-2018.tar.gz


If you are curious how I did this, please check out Setup_Chapter6.ipynb. No need to run this or edit it or even look at it, but it's there.

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark") \
    .getOrCreate()

sc = spark.sparkContext

23/11/07 21:16:20 WARN Utils: Your hostname, classes resolves to a loopback address: 127.0.1.1; using 192.168.122.111 instead (on interface enp1s0)
23/11/07 21:16:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/07 21:16:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/07 21:16:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/11/07 21:16:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/11/07 21:16:21 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/11/07 21:16:21 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/11/07 21:16:21 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 40

## Exploratory Data Analysis

In [6]:
# Our main data source
on_time_df = spark.read.parquet('file:///disk/airline-data-processed/airline-data.parquet')
on_time_df.show()

23/11/07 21:16:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----------+---------+----------+-------------+---------+-------+-------+---------+---------------+------------------+------------------+------+---------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+-----------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+----+-----+
|Quarter|DayofMonth|DayOfWeek|FlightDate|UniqueCarrier|AirlineID|Carrier|TailNum|FlightNum|OriginAirportID|OriginAirportSeqID|OriginCityMarketID|Origin| OriginCityName|OriginState|OriginStateFips|OriginStateName|OriginWac|DestAirportID|DestAirportSeqID|DestCityMarketID|De

That is a bit brutal to look at... Consider examining like:

In [7]:
on_time_df.columns

['Quarter',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'UniqueCarrier',
 'AirlineID',
 'Carrier',
 'TailNum',
 'FlightNum',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'Year',
 'Month']

In [8]:
# The first row
on_time_df.first()

Row(Quarter=3, DayofMonth=5, DayOfWeek=3, FlightDate=datetime.date(2018, 9, 5), UniqueCarrier='MQ', AirlineID=20398, Carrier='MQ', TailNum='N908AE', FlightNum=3492, OriginAirportID=13930, OriginAirportSeqID=1393006, OriginCityMarketID=30977, Origin='ORD', OriginCityName='Chicago, IL', OriginState='IL', OriginStateFips=17, OriginStateName='Illinois', OriginWac=41, DestAirportID=13367, DestAirportSeqID=1336705, DestCityMarketID=33367, Dest='MLI', DestCityName='Moline, IL', DestState='IL', DestStateFips=17, DestStateName='Illinois', DestWac=41, CRSDepTime='0840', DepTime='0834', DepDelay=-6.0, DepDelayMinutes=0.0, DepDel15=0.0, DepartureDelayGroups=-1, DepTimeBlk='0800-0859', TaxiOut=14.0, WheelsOff='0848', WheelsOn='0915', TaxiIn=5.0, CRSArrTime='0935', ArrTime='0920', ArrDelay=-15.0, ArrDelayMinutes=0.0, ArrDel15=0.0, ArrivalDelayGroups=-1, ArrTimeBlk='0900-0959', Cancelled=0.0, CancellationCode=None, Diverted=0.0, CRSElapsedTime=55.0, ActualElapsedTime=46.0, AirTime=27.0, Flights=1.0, 

What if you want to average AirTime?

In [9]:
from pyspark.sql.functions import avg, col

on_time_df.select('AirTime').agg(
    avg(col('AirTime'))
).show()

+----------------+
|    avg(AirTime)|
+----------------+
|106.840895516509|
+----------------+



So we need navigate a fine line where I don't throw the entire Spark SQL API at you, but there are some functions above that should be discussed. The first is select which you can use to get a subset of the columns. This is important for memory usage. Load only what you need :). The next few are agg which is short for aggregate. Then there is col which selects the column and then avg which of course is average. If you know sql, you can also rely on SQL to work the magic.

In [10]:
on_time_df.select('AirTime').createOrReplaceTempView("AirTimeView") # create a temporary view so we can query our data

sqlDF = spark.sql("SELECT avg(AirTime) FROM AirTimeView").show()

+----------------+
|    avg(AirTime)|
+----------------+
|106.840895516509|
+----------------+



I don't know about you, but since I already know SQL or at least some SQL, I'm very excited that I can use that. For this topic in general, please use what makes sense to you to accomplish the job.

What if I wanted average air time per month?

In [11]:
on_time_df.select('AirTime','Month').createOrReplaceTempView("AirTimeView") # create a temporary view so we can query our data

sqlDF = spark.sql("SELECT Month, avg(AirTime) FROM AirTimeView group by Month").show()

+-----+------------------+
|Month|      avg(AirTime)|
+-----+------------------+
|    9|105.11722965450537|
|    6|106.96005748333293|
|    4|106.98955608533186|
|    3|108.13478826356425|
|    8|106.53259036679218|
|    7|107.72642114700079|
|    5|106.01523155420969|
|   11|106.60017004471732|
|   12| 108.9605027894764|
|    1|106.82890250362068|
|   10|105.03766685659184|
|    2|107.08287579839748|
+-----+------------------+



In [12]:
on_time_df.select('AirTime','Month').groupBy(
    'Month'
).agg(
    avg(col('AirTime'))
).show()

+-----+------------------+
|Month|      avg(AirTime)|
+-----+------------------+
|    9|105.11722965450537|
|    6|106.96005748333293|
|    4|106.98955608533186|
|    3|108.13478826356425|
|    8|106.53259036679218|
|    7|107.72642114700079|
|    5|106.01523155420969|
|   11|106.60017004471732|
|   12| 108.9605027894764|
|    1|106.82890250362068|
|   10|105.03766685659184|
|    2|107.08287579839748|
+-----+------------------+



Pretty nice right? You can see why companies might really value engineers who can bring data processing skills with them.

Let's now read in some data that helps us map Carrier name to AirlineName.

In [13]:
airlines = spark.read.parquet('file:///disk/airline-data/DOT_airline_codes_table')

In [14]:
airlines.show()

+---------+--------------------+-------+
|AirlineID|         AirlineName|Carrier|
+---------+--------------------+-------+
|    19031|Mackey Internatio...|    MAC|
|    19032|Munz Northern Air...|     XY|
|    19033|Cochise Airlines ...|    COC|
|    19034|Golden Gate Airli...|    GSA|
|    19035|       Aeromech Inc.|    RZZ|
|    19036|Golden West Airli...|    GLW|
|    19037|Puerto Rico Intl ...|    PRN|
|    19038|    Air America Inc.|    STZ|
|    19039|Swift Aire Lines ...|    SWT|
|    19040|American Central ...|    TSF|
|    19041|     Valdez Airlines|    VEZ|
|    19042|Southeast Alaska ...|    WEB|
|    19043|Altair Airlines Inc.|    AAR|
|    19044| Chitina Air Service|    CHI|
|    19045|Marco Island Airw...|    MRC|
|    19046|Caribbean Air Ser...|    OHZ|
|    19047|   Sundance Airlines|    PRO|
|    19048|Seair Alaska Airl...|    SAI|
|    19049|Southeast Airline...|    SLZ|
|    19050|Alaska Aeronautic...|    AAZ|
+---------+--------------------+-------+
only showing top

What if we want to apply a user defined function? Here is an example where a new function is defined that combines Year and Month into a string. I also use the sample function to illustrate how to get a random subset of the data. Finally, I show an important function called ``cache``. It is important because we may want to reuse a result. Cache tells Spark that we want to reuse something so please try to keep it cached for us. Finally, I show how you can use orderBy to sort the data.

In [15]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def getYearMonthStr(year, month):
    return '%d-%02d'%(year,month)

udfGetYearMonthStr = udf(getYearMonthStr, StringType())

example1 = on_time_df.select('Year','Month').withColumn(
    'YearMonth', udfGetYearMonthStr('Year','Month')).sample(0.000001).cache()

example1.show()

+----+-----+---------+
|Year|Month|YearMonth|
+----+-----+---------+
|2018|    9|  2018-09|
|2018|    9|  2018-09|
|2004|    6|  2004-06|
|2004|    6|  2004-06|
|2017|    4|  2017-04|
|2017|    4|  2017-04|
|2016|    3|  2016-03|
|2016|    3|  2016-03|
|2018|    8|  2018-08|
|2018|    7|  2018-07|
|2018|   11|  2018-11|
|2006|    8|  2006-08|
|2006|    8|  2006-08|
|2007|    8|  2007-08|
|2006|    1|  2006-01|
|2007|    3|  2007-03|
|2004|    8|  2004-08|
|2004|    8|  2004-08|
|2018|    1|  2018-01|
|2018|    4|  2018-04|
+----+-----+---------+
only showing top 20 rows



In [16]:
example1.orderBy('YearMonth').show()

+----+-----+---------+
|Year|Month|YearMonth|
+----+-----+---------+
|2003|    3|  2003-03|
|2003|    5|  2003-05|
|2003|    7|  2003-07|
|2003|    9|  2003-09|
|2003|   10|  2003-10|
|2004|    1|  2004-01|
|2004|    1|  2004-01|
|2004|    2|  2004-02|
|2004|    3|  2004-03|
|2004|    6|  2004-06|
|2004|    6|  2004-06|
|2004|    8|  2004-08|
|2004|    8|  2004-08|
|2004|   10|  2004-10|
|2004|   11|  2004-11|
|2004|   11|  2004-11|
|2004|   11|  2004-11|
|2005|    1|  2005-01|
|2005|    2|  2005-02|
|2005|    3|  2005-03|
+----+-----+---------+
only showing top 20 rows



Finally, there are a number of things to make the world go round, such as renaming a column:
```python
df.withColumnRenamed("dob","DateOfBirth").printSchema()
```

In [17]:
def getYearMonthStr(year, month):
    return '%d-%02d'%(year,month)

udfGetYearMonthStr = udf(getYearMonthStr, StringType())

**Exercise 1:** Create a dataframe that contains the average delay for each airline for each month of each year (i.e., group by carrier, year, and month):
* Columns: Carrier, average_delay, YearMonth
* Carrier must be one of the following: 'AA','WN','DL','UA','MQ','EV','AS','VX'
* Must be ordered by YearMonth, Carrier
* The column to aggregate is ArrDelay

In [18]:
def exercise_1(on_time_df):
    result = None
    # Your solution here
    return result

In [19]:
airline_delay = exercise_1(on_time_df)
airline_delay.show()

+-------+-------------------+---------+
|Carrier|      average_delay|YearMonth|
+-------+-------------------+---------+
|     AA|-1.7389585107677332|  2003-01|
|     AS| 0.9330533596837944|  2003-01|
|     DL| 0.5961176570314487|  2003-01|
|     EV|  4.787899330565207|  2003-01|
|     MQ|  3.932421379213087|  2003-01|
|     UA|-1.0406547214673914|  2003-01|
|     WN| 0.9824094678087045|  2003-01|
|     AA| 2.6314517551596404|  2003-02|
|     AS|  4.260234991738572|  2003-02|
|     DL|  5.211223847726915|  2003-02|
|     EV|  9.873830172962908|  2003-02|
|     MQ|    7.2193594912643|  2003-02|
|     UA| 0.6762339377927225|  2003-02|
|     WN|  5.340074655969692|  2003-02|
|     AA|-2.0331744783232315|  2003-03|
|     AS|  1.960201429499675|  2003-03|
|     DL| 1.8281763060511822|  2003-03|
|     EV|  9.114690219509027|  2003-03|
|     MQ|  1.166732684472612|  2003-03|
|     UA|0.24932108018857677|  2003-03|
+-------+-------------------+---------+
only showing top 20 rows



**Exercise 2:** Now add a column with the airline name (i.e., use a join). Here is an example from the Spark documentation. Please order your result by YearMonth and Carrier.

```python
# To create DataFrame using SparkSession
people = spark.read.parquet("...")
department = spark.read.parquet("...")

people.filter(people.age > 30).join(department, people.deptId == department.id) \
  .groupBy(department.name, "gender").agg({"salary": "avg", "age": "max"})
```

In [20]:
def exercise_2(airline_delay,airlines):
    result = None
    # Your solution here
    return result

In [21]:
airline_delay2 = exercise_2(airline_delay,airlines)
airline_delay2.show()

+-------+-------------------+---------+--------------------+
|Carrier|      average_delay|YearMonth|         AirlineName|
+-------+-------------------+---------+--------------------+
|     AA|-1.7389585107677332|  2003-01|American Airlines...|
|     AS| 0.9330533596837944|  2003-01|Alaska Airlines Inc.|
|     DL| 0.5961176570314487|  2003-01|Delta Air Lines Inc.|
|     EV|  4.787899330565207|  2003-01|ExpressJet Airlin...|
|     MQ|  3.932421379213087|  2003-01|           Envoy Air|
|     UA|-1.0406547214673914|  2003-01|United Air Lines ...|
|     WN| 0.9824094678087045|  2003-01|Southwest Airline...|
|     AA| 2.6314517551596404|  2003-02|American Airlines...|
|     AS|  4.260234991738572|  2003-02|Alaska Airlines Inc.|
|     DL|  5.211223847726915|  2003-02|Delta Air Lines Inc.|
|     EV|  9.873830172962908|  2003-02|ExpressJet Airlin...|
|     MQ|    7.2193594912643|  2003-02|           Envoy Air|
|     UA| 0.6762339377927225|  2003-02|United Air Lines ...|
|     WN|  5.34007465596

If you did everything correctly, you are now rewarded with a nice graph :)

In [22]:
import numpy as np

airline_delay_pd = airline_delay2.toPandas()

import altair as alt

alt.Chart(airline_delay_pd).mark_line().encode(
    x='YearMonth',
    y='average_delay',
    color='AirlineName'
)

alt.Chart(...)

**Exercise 3:** Let's assume you believe that the delays experienced by some airlines are correlated. The cause is a different story as we all know correlation does not equal causation. But correlation is often what we can easily calculate, so let's do it on a month by month basis. The first step is of course to get the data in the correct format. We would like each airline to have it's own column because we can easily compute the correlation between columns. Each row in this new dataframe should be a YearMonth.

In [23]:
def exercise_3(airline_delay2):
    result = None
    # partial solution
    # airline_delay2.groupBy(?).pivot(?).agg(avg(?))
    # Your solution here
    return result

In [24]:
data_for_corr = exercise_3(airline_delay2)

# The data is now small enough to handle, so let's get it into pandas and calculate the correlation and filling
# in missing values with the mean of the column

df = data_for_corr.toPandas().set_index('YearMonth')
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(df)

df_imputed_nan = pd.DataFrame(imp_mean.transform(df),columns=df.columns,index=df.index)
df_imputed_nan

,Alaska Airlines Inc.,American Airlines Inc.,Delta Air Lines Inc.,Envoy Air,ExpressJet Airlines LLC,Southwest Airlines Co.,United Air Lines Inc.,Virgin America
YearMonth,,,,,,,,
2013-05,-1.956498,8.391558,0.622329,13.622002,9.923755,6.523672,3.067576,6.940016
2009-07,-0.123909,11.631664,10.045305,6.516201,16.259290,4.412852,3.810207,5.828086
2013-09,-0.679653,1.241006,-2.208259,2.129175,1.616862,7.767035,-0.186396,2.414229
2010-08,-2.173459,4.173351,6.004868,5.484603,4.003977,3.635669,-3.599980,5.828086
2007-05,6.805009,12.350215,1.966700,10.732516,5.618201,3.715613,8.502022,5.828086
...,...,...,...,...,...,...,...,...
2009-02,6.387049,1.757281,0.743976,3.207515,5.054657,-2.871507,0.485592,5.828086
2015-03,-0.544731,6.693587,2.077965,11.018418,6.730101,3.263341,4.892212,5.348207
2007-04,3.604306,13.073769,3.972544,10.661143,11.056802,3.389504,11.012593,5.828086


Now let's take a look at the correlations

In [25]:
df_imputed_nan.corr()

,Alaska Airlines Inc.,American Airlines Inc.,Delta Air Lines Inc.,Envoy Air,ExpressJet Airlines LLC,Southwest Airlines Co.,United Air Lines Inc.,Virgin America
Alaska Airlines Inc.,1.000000,0.464925,0.557909,0.348146,0.391331,0.235272,0.529354,0.157809
American Airlines Inc.,0.464925,1.000000,0.617717,0.686899,0.556708,0.566585,0.789016,0.136637
Delta Air Lines Inc.,0.557909,0.617717,1.000000,0.516147,0.640598,0.495264,0.531381,0.203032
Envoy Air,0.348146,0.686899,0.516147,1.000000,0.517757,0.609208,0.621563,0.173870
ExpressJet Airlines LLC,0.391331,0.556708,0.640598,0.517757,1.000000,0.490620,0.571136,0.132579
Southwest Airlines Co.,0.235272,0.566585,0.495264,0.609208,0.490620,1.000000,0.541756,0.220976
United Air Lines Inc.,0.529354,0.789016,0.531381,0.621563,0.571136,0.541756,1.000000,0.194124
Virgin America,0.157809,0.136637,0.203032,0.173870,0.132579,0.220976,0.194124,1.000000


**Stop and think:** What stands out to you? Let me clean it up and sort it for you.

In [26]:
corrs = df_imputed_nan.corr()
corrs.values[np.tril_indices(len(corrs))] = np.NaN 
corrs.stack().sort_values(ascending=False)

American Airlines Inc.   United Air Lines Inc.      0.789016
                         Envoy Air                  0.686899
Delta Air Lines Inc.     ExpressJet Airlines LLC    0.640598
Envoy Air                United Air Lines Inc.      0.621563
American Airlines Inc.   Delta Air Lines Inc.       0.617717
Envoy Air                Southwest Airlines Co.     0.609208
ExpressJet Airlines LLC  United Air Lines Inc.      0.571136
American Airlines Inc.   Southwest Airlines Co.     0.566585
Alaska Airlines Inc.     Delta Air Lines Inc.       0.557909
American Airlines Inc.   ExpressJet Airlines LLC    0.556708
Southwest Airlines Co.   United Air Lines Inc.      0.541756
Delta Air Lines Inc.     United Air Lines Inc.      0.531381
Alaska Airlines Inc.     United Air Lines Inc.      0.529354
Envoy Air                ExpressJet Airlines LLC    0.517757
Delta Air Lines Inc.     Envoy Air                  0.516147
                         Southwest Airlines Co.     0.495264
ExpressJet Airlines LLC 

In [11]:
# Don't forget to push with ./submit.sh